In [3]:
import pandas as pd
from langchain_core.tools import tool
import yfinance as yf
from ta.momentum import RSIIndicator , StochasticOscillator
from ta.trend import SMAIndicator , EMAIndicator , MACD
from ta.volume import volume_weighted_average_price
import datetime as dt

In [4]:
#Creating a tool

@tool
def get_stock_prices( ticker : str ) :
    """This tool fetches historical stock prices data and technical indicator for a given stock"""

    try:
        data = yf.download( ticker , start = dt.datetime.now() - dt.timedelta( weeks = 24*3 ) , end = dt.datetime.now() , interval = "1wk")
        df = data.copy()
        data.reset_index( inplace = True )
        data.Date = data.Date.astype(str)
        indicators = {}

        rsi_series = RSIIndicator(df["Close"], window=14).rsi().iloc[-12:]
        indicators["RSI"] = {date.strftime("%Y-%m-%d"): int(value) for date , value in rsi_series.dropna().to_dict().items()}

        sto_series = StochasticOscillator(df['High'], df['Low'], df['Close'], window=14).stoch().iloc[-12:]
        indicators["Stochastic_Oscillator"] = {
                    date.strftime('%Y-%m-%d'): int(value) 
                    for date, value in sto_series.dropna().to_dict().items()}

        macd = MACD(df['Close'])
        macd_series = macd.macd().iloc[-12:]
        indicators["MACD"] = {date.strftime('%Y-%m-%d'): int(value) 
                    for date, value in macd_series.to_dict().items()}
        
        macd_signal_series = macd.macd_signal().iloc[-12:]
        indicators["MACD_Signal"] = {date.strftime('%Y-%m-%d'): int(value) 
                    for date, value in macd_signal_series.to_dict().items()}
        
        vwap_series = volume_weighted_average_price(
            high=df['High'], low=df['Low'], close=df['Close'], 
            volume=df['Volume'],
        ).iloc[-12:]
        indicators["vwap"] = {date.strftime('%Y-%m-%d'): int(value) 
                    for date, value in vwap_series.to_dict().items()}
        
        return {'stock_price': data.to_dict(orient='records'),
                'indicators': indicators}

    except Exception as e:
        return f"Error fetching price data: {str(e)}"





In [5]:

@tool
def get_financial_metrics(ticker: str) :
    """This tool Fetches key financial ratios for a given ticker."""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            'pe_ratio': info.get('forwardPE'),
            'price_to_book': info.get('priceToBook'),
            'debt_to_equity': info.get('debtToEquity'),
            'profit_margins': info.get('profitMargins')
        }
    except Exception as e:
        return f"Error fetching ratios: {str(e)}"

In [6]:
from langgraph.graph import StateGraph , START , END , add_messages
from typing import TypedDict , Annotated

class State( TypedDict ):

    messages:Annotated[list , add_messages]
    stock:str


graph_builder = StateGraph(State)

In [7]:
from dotenv import load_dotenv
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

load_dotenv()

llm = ChatGoogleGenerativeAI( model = "gemini-2.0-flash" )

tools = [ get_stock_prices , get_financial_metrics ]
binded_llm = llm.bind_tools( tools )



In [25]:
FUNDAMENTAL_ANALYST_PROMPT = """
You are a fundamental analyst specializing in evaluating company (whose symbol is {company}) performance based on stock prices, technical indicators, and financial metrics. Your task is to provide a comprehensive summary of the fundamental analysis for a given stock.

You have access to the following tools:
1. **get_stock_prices**: Retrieves the latest stock price, historical price data and technical Indicators like RSI, MACD, Drawdown and VWAP.
2. **get_financial_metrics**: Retrieves key financial metrics, such as revenue, earnings per share (EPS), price-to-earnings ratio (P/E), and debt-to-equity ratio.

### Your Task:
1. **Input Stock Symbol**: Use the provided stock symbol to query the tools and gather the relevant information.
2. **Analyze Data**: Evaluate the results from the tools and identify potential resistance, key trends, strengths, or concerns.
3. **Provide Summary**: Write a concise, well-structured summary that highlights:
    - Recent stock price movements, trends and potential resistance.
    - Key insights from technical indicators (e.g., whether the stock is overbought or oversold).
    - Financial health and performance based on financial metrics.

### Constraints:
- Use only the data provided by the tools.
- Avoid speculative language; focus on observable data and trends.
- If any tool fails to provide data, clearly state that in your summary.

### Output Format:
Respond in the following format:
"stock": "<Stock Symbol>",
"price_analysis": "<Detailed analysis of stock price trends>",
"technical_analysis": "<Detailed time series Analysis from ALL technical indicators>",
"financial_analysis": "<Detailed analysis from financial metrics>",
"final Summary": "<Full Conclusion based on the above analyses>"
"Asked Question Answer": "<Answer based on the details and analysis above>"

Ensure that your response is objective, concise, and actionable."""

In [26]:
from langchain_core.messages import SystemMessage

def fundamental_analyst( state : State ):

    messages = [
        SystemMessage(content = FUNDAMENTAL_ANALYST_PROMPT.format(company=state["stock"])),
    ] + state["messages"]
    return {
        "messages":binded_llm.invoke(messages)
    }


In [27]:
def tools_condition( state ):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [28]:
from langgraph.prebuilt import ToolNode


# Reset the Graph to avoid duplicate nodes
graph_builder = StateGraph(State)

# Add Nodes
if "fundamental_analyst" not in graph_builder.nodes:
    graph_builder.add_node("fundamental_analyst", fundamental_analyst)

graph_builder.add_edge(START, "fundamental_analyst")

if "tools" not in graph_builder.nodes:
    graph_builder.add_node("tools", ToolNode(tools=tools))

if "fundamental_analyst" in graph_builder.branches:
    if "tools_condition" not in graph_builder.branches["fundamental_analyst"]:
        graph_builder.add_conditional_edges("fundamental_analyst", tools_condition)
else:
    graph_builder.add_conditional_edges("fundamental_analyst", tools_condition)

if "tools" in graph_builder.nodes and ("tools", "fundamental_analyst") not in graph_builder.edges:
    graph_builder.add_edge("tools", "fundamental_analyst")

# Compile the Graph
graph = graph_builder.compile()
print("Graph compiled successfully ✅")


Graph compiled successfully ✅


In [29]:
for event in graph.stream( { "messages" : [ ("user","Is it ok to purchase google stocks" ) ] , "stock":"google" } , stream_mode="values" ):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Is it ok to purchase google stocks
================================== Ai Message ==================================
Tool Calls:
  get_stock_prices (7284962e-cdea-4245-8e95-950bb7307491)
 Call ID: 7284962e-cdea-4245-8e95-950bb7307491
  Args:
    ticker: GOOGL
  get_financial_metrics (a398109a-1340-4eef-ab9f-b9e66cc8fc77)
 Call ID: a398109a-1340-4eef-ab9f-b9e66cc8fc77
  Args:
    ticker: GOOGL
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


================================= Tool Message =================================
Name: get_financial_metrics

{"pe_ratio": 19.421875, "price_to_book": 6.5366993, "debt_to_equity": 8.655, "profit_margins": 0.28604}
================================== Ai Message ==================================

"stock": "GOOGL",
"price_analysis": "Price data could not be retrieved due to an error.",
"technical_analysis": "Technical indicator data could not be retrieved due to an error.",
"financial_analysis": "The debt-to-equity ratio is 8.655, the P/E ratio is 19.42, the price-to-book ratio is 6.54, and the profit margin is 0.29.",
"final Summary": "I was unable to retrieve stock price and technical indicator data. Based on available financial metrics, Google has a debt-to-equity ratio of 8.655, a P/E ratio of 19.42, a price-to-book ratio of 6.54 and a profit margin of 29%.",
"Asked Question Answer": "I am unable to provide a purchase recommendation without stock price data and technical indicator ana